In [ ]:
%%capture
!pip install wilds
!apt-get install libmagickwand-dev
!pip install wand

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!mkdir /content/data/
!mkdir /content/data/camelyon17_v1.0/
!tar -xvzf /content/drive/MyDrive/clear-vae/data/camelyon17_v1.0/archive.tar.gz -C /content/data/camelyon17_v1.0/

In [ ]:
import sys
import json

sys.path.append('/content/drive/MyDrive/clear-vae')

In [ ]:
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
dataset = get_dataset(dataset="camelyon17", download=False)

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
train_data = dataset.get_subset(
    "train",
    transform=transforms.Compose(
        [transforms.Resize((64, 64)), transforms.ToTensor()]
    ),
)
train_data, valid_data = random_split(train_data, [0.8, 0.2])

test_data = dataset.get_subset(
    "test",
    transform=transforms.Compose(
        [transforms.Resize((64, 64)), transforms.ToTensor()]
    ),
)

def collate_fn(batch):
    x, y, meta = zip(*batch)
    return torch.stack(x), torch.stack(y), torch.stack(meta)[:,0]

train_loader = DataLoader(train_data, batch_size=64, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, collate_fn=collate_fn)

In [ ]:
import torch.nn as nn
import numpy as np
from src.trainer import DownstreamMLPTrainer, SimpleCNNTrainer
from src.utils.trainer_utils import *
import os

In [ ]:
def init_weights(module):
    if isinstance(module, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
        # He uniform initialization
        torch.nn.init.kaiming_uniform_(module.weight, mode='fan_in', nonlinearity='relu')
        if module.bias is not None:
            torch.nn.init.zeros_(module.bias)

In [ ]:
def experiment_helper(
    dataloaders,
    vae_trainer,
    epochs,
):
    train_loader = dataloaders["train_loader"]
    valid_loader = dataloaders["valid_loader"]
    test_loader = dataloaders["test_loader"]
    # Train VAE
    vae_trainer.fit(epochs, train_loader, valid_loader)
    vae_trainer.model.eval()

    z_dim = vae_trainer.model.z_dim  # which is 1/2 * total_z_dim
    device = vae_trainer.device
    # Freeze VAE parameters
    for p in vae_trainer.model.parameters():
        p.requires_grad = False

    # Create and train MLP classifier
    mlp = nn.Sequential(
        nn.Linear(z_dim, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Linear(256, 10),
    ).to(device)
    optimizer = torch.optim.Adam(mlp.parameters(), lr=3e-4)
    criterion = nn.CrossEntropyLoss()

    trainer = DownstreamMLPTrainer(
        vae_trainer.model, mlp, optimizer, criterion, 1, device
    )
    trainer.fit(1, train_loader, valid_loader)

    # Evaluate on test set
    (aupr_scores, auroc_scores), acc = trainer.evaluate(test_loader, False, 0)

    return aupr_scores, auroc_scores, acc


def experiment(dataloaders, trainer_kwargs, epochs, run_index,
               save_root="/content/drive/MyDrive/clear-vae/expr_output/camelyon17/result"):
    train_loader = dataloaders["train_loader"]
    valid_loader = dataloaders["valid_loader"]
    test_loader = dataloaders["test_loader"]

    tc_trainer_kwargs = trainer_kwargs.copy()
    tc_trainer_kwargs["temperature"] = 0.1

    models = {
        "baseline": (
            get_cnn_trainer,
            {"n_class": 2, "device": trainer_kwargs["device"], "cnn_arch": "SimpleCNN64Classifier",
             "in_channel": trainer_kwargs["in_channel"], "verbose_period": trainer_kwargs["verbose_period"]},
        ),
        "clear": (
            get_clearvae_trainer,
            {"ps": True, **trainer_kwargs},
        ),
        "clear-mim (L1OutUB)": (
            get_clearmimvae_trainer,
            {
                "mi_estimator": "L1OutUB",
                "la": 3,
                "mi_estimator_lr": 2e-3,
                **trainer_kwargs,
            },
        ),
        "clear-mim (CLUB-S)": (
            get_clearmimvae_trainer,
            {
                "mi_estimator": "CLUBSample",
                "la": 3,
                "mi_estimator_lr": 2e-3,
                **trainer_kwargs,
            },
        ),
        "clear-tc": (
            get_cleartcvae_trainer,
            {"la": 1, "factor_cls_lr": 5e-4, **tc_trainer_kwargs},
        ),
        "lamcnn": (
            get_lamcnn_trainer,
            {"n_class": 2, "lam_coef": 0.001,
             "device": trainer_kwargs["device"], "cnn_arch": "LAMCNN64Classifier",
             "in_channel": trainer_kwargs["in_channel"], "verbose_period": trainer_kwargs["verbose_period"]},
        ),
        "gvae": (
            get_hierarchical_vae_trainer,
            {
                "beta": trainer_kwargs["beta"],
                "vae_lr": 1e-4,
                "vae_arch": trainer_kwargs["vae_arch"],
                "z_dim": trainer_kwargs["z_dim"],
                "group_mode": "GVAE",
                "device": trainer_kwargs["device"],
                "in_channel": trainer_kwargs["in_channel"],
                "verbose_period": trainer_kwargs["verbose_period"],
            },
        ),
        "mlvae": (
            get_hierarchical_vae_trainer,
            {
                "beta": trainer_kwargs["beta"],
                "vae_lr": 1e-4,
                "vae_arch": trainer_kwargs["vae_arch"],
                "z_dim": trainer_kwargs["z_dim"],
                "group_mode": "MLVAE",
                "device": trainer_kwargs["device"],
                "in_channel": trainer_kwargs["in_channel"],
                "verbose_period": trainer_kwargs["verbose_period"],
            },
        ),
    }

    filename = f"{save_root}_{run_index}.json"
    if os.path.exists(filename):
        with open(filename, "r") as infile:
            results = json.load(infile) # Load existing data
    else:
        results = {}

    for model_name, (trainer_func, params) in models.items():
        print(f"\nTraining {model_name}:")
        trainer = trainer_func(**params)
        trainer.model.apply(init_weights)

        if isinstance(trainer, SimpleCNNTrainer):
            trainer.fit(
                epochs=6, train_loader=train_loader, valid_loader=valid_loader
            )
            (aupr_scores, auroc_scores), acc = trainer.evaluate(test_loader, False, 0)
        else:
            aupr_scores, auroc_scores, acc = experiment_helper(
                dataloaders, trainer, epochs
            )
        temp = {
            "acc": round(float(acc), 3),
            "pr": {
                "overall": round(np.mean(list(aupr_scores.values())), 3),
                "stratified": aupr_scores,
            },
            "roc": {
                "overall": round(np.mean(list(auroc_scores.values())), 3),
                "stratified": auroc_scores,
            },
        }

        results[model_name] = temp
        with open(filename, "w") as outfile:
            json.dump(results, outfile, indent=4)

    return results

In [ ]:
dataloaders = {
    "train_loader": train_loader,
    "valid_loader": valid_loader,
    "test_loader": test_loader,
}
trainer_kwargs = {
    "in_channel": 3,
    "vae_arch": "VAE64",
    "beta": 1 / 32,
    "vae_lr": 1e-4,
    "z_dim": 64,
    "alpha": 100,
    "temperature": 0.3,
    "device": device,
    "verbose_period": 2
}

In [15]:
for i in range(1):
    s = np.random.randint(1000)
    np.random.seed(s)
    torch.manual_seed(s)
    rlt = experiment(dataloaders, trainer_kwargs, 7, run_index=s)



Training baseline:


val-epoch 0: 100%|██████████| 946/946 [01:03<00:00, 14.82it/s]


val_aupr: {0: np.float64(0.99), 1: np.float64(0.992)}
0.991
val_auroc: {0: np.float64(0.991), 1: np.float64(0.991)}
0.991
val_acc: 0.958


val-epoch 2: 100%|██████████| 946/946 [01:03<00:00, 14.88it/s]


val_aupr: {0: np.float64(0.994), 1: np.float64(0.996)}
0.995
val_auroc: {0: np.float64(0.995), 1: np.float64(0.995)}
0.995
val_acc: 0.968


...

val_recontr_loss=182.324, val_kl_c=333151200.000, val_kl_s=81.579
gMIG: 0.018; mse: 182.324


val-epoch 6: 100%|██████████| 946/946 [01:04<00:00, 14.72it/s]


val_recontr_loss=172.421, val_kl_c=50106556.000, val_kl_s=78.458
gMIG: 0.025; mse: 172.421


val-epoch 0: 100%|██████████| 946/946 [01:02<00:00, 15.02it/s]


val_aupr: {0: np.float64(0.933), 1: np.float64(0.932)}
0.933
val_auroc: {0: np.float64(0.933), 1: np.float64(0.933)}
0.933
val_acc: 0.859


In [ ]:
from tqdm import tqdm
import time

for i in tqdm(range(9)):
    time.sleep(1)

!kill $(ps aux | awk '{print $2}')

100%|██████████| 9/9 [00:09<00:00,  1.00s/it]